# Using Gradio to Deploy Machine Learning Models


Time estimate: **45** minutes

## Objectives

After completing this lab, you will be able to:
- Create interactive web interfaces for machine learning models using Gradio
- Train a classification model to categorize individuals based on BMI (Body Mass Index)
- Save and load trained models using joblib for deployment
- Build a user-friendly application that healthcare professionals can use without coding knowledge

## What you will do in this lab

In this lab, you will:
- Understand the basics with simple Gradio apps
- Generate a synthetic dataset of height and weight measurements
- Train a machine learning model to classify individuals into weight categories (Underweight, Normal Weight, Obese)
- Save your trained model for future use
- Create an interactive Gradio web application that loads your model and makes predictions

## Overview

In modern healthcare, machine learning models are increasingly being used to assist with clinical decision-making. However, these models are often developed by data scientists and stored as code that clinicians cannot easily access or use. This creates a significant gap between model development and clinical application.

Gradio is a Python library that bridges this gap by allowing developers to quickly create user-friendly web interfaces for machine learning models. With just a few lines of code, you can transform a complex model into an interactive application that healthcare professionals can use directly in their workflow—no programming knowledge required.

In this lab, you'll focus on a practical healthcare scenario: classifying individuals into weight categories based on their height and weight measurements. While BMI classification is a simplified example, the techniques you learn here apply to more complex medical applications, such as disease prediction, medical image analysis, and patient risk stratification. Understanding how to deploy models effectively is just as important as building them; it ensures your work can actually benefit patients and clinicians in real-world settings.

## About the dataset

For this lab, you will create a **synthetic dataset** containing height and weight measurements. This dataset is simulated for educational purposes and mimics realistic distributions of human anthropometric data.

**Features:**
- **Height (cm)**: Continuous variable ranging from 150 to 200 cm
- **Weight (kg)**: Continuous variable ranging from 40 to 120 kg

**Target variable:**
- **BMI Category**: Categorical variable with three classes:
  - **Underweight**: BMI < 18.5
  - **Normal Weight**: 18.5 ≤ BMI < 30
  - **Obese**: BMI ≥ 30

Note: BMI is calculated as weight (kg) / (height (m))²

## Setup

First, you'll install and import all necessary libraries for this lab.

In [ ]:
# Install required packages
!pip install gradio pandas scikit-learn matplotlib seaborn joblib numpy

In [ ]:
# Import necessary libraries for data manipulation
import pandas as pd
import numpy as np

# Import libraries for machine learning
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Import libraries for visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Import Gradio for building interactive interfaces
import gradio as gr

# Import joblib for saving and loading models
import joblib

# Set random seed for reproducibility
np.random.seed(42)

## Step 1: Your first Gradio app - A simple greeter

Before you dive into machine learning, understand how Gradio works with an ultra-simple example. You'll create a basic application that greets users by name.

In [ ]:
# Define a simple function that takes a name and returns a greeting
def greet(name):
    return f"Hello, {name}! Welcome to Gradio."

# Create a Gradio interface
# - fn: the function to call
# - inputs: type of input widget ("text" creates a text box)
# - outputs: type of output widget ("text" displays text)
demo = gr.Interface(
    fn=greet,
    inputs="text",
    outputs="text"
)

# Launch the interface
# share=False keeps the app local (set to True to create a public link)
demo.launch(share=False)

**What just happened?**

You created a web interface in just a few lines of code! Here's the breakdown:
1. You defined a Python function (`greet`) that takes input and returns output.
2. You wrapped it with `gr.Interface()` specifying what types of inputs and outputs to use.
3. You launched it with `.launch()` to create a live web app.

This same pattern applies to machine learning models; the only difference is that your function will call a trained model instead of returning a simple string.

## Step 2: Second Gradio app - Compute the square of a number

 You'll create a basic application that calculates the square of a number.

In [ ]:
# Define a simple function that takes a number and returns its square
def square(num):
    return num ** 2

# Create a Gradio interface
# - fn: the function to call
# - inputs: type of input widget ("slider" creates a slider to input a number)
# - outputs: type of output widget ("Number" displays numeric values)
# - title will let you set the title of the application
# - descriptions allows a short description to be displayed

demo2 = gr.Interface(
    fn=square,
    inputs=gr.Slider(minimum=0, maximum=100, step=1, label="Select a number"),
    outputs=gr.Number(),
    title="Square Calculator",
    description="Move the slider to get the square of the number."
)

demo2.launch(share=False)


## Step 3: Third Gradio app - Word count program

This program will count the number of words in the text that you have entered.

In [ ]:
# Define a simple function that takes a piece of text and and returns number of words

def word_count(text):
    words = text.split()
    return len(words)

# Create a Gradio interface
# - fn: the function to call
# - inputs: type of input widget ("textbox" creates a multiline text box)
# - outputs: type of output widget ("text" displays text)
# - title will let you set the title of the application
# - descriptions allows a short description to be displayed

demo2 = gr.Interface(
    fn=word_count,
    inputs=gr.Textbox(lines=5, label="Enter your paragraph here"),
    outputs=gr.Textbox(lines=1, label="Word count"),
    submit_btn="Get Word Count",  # Custom button text
    title="Simple Word Counter",
    description="Counts the number of words in the given text."
)

demo2.launch(share=False)


## Step 4: Generate and explore the height-weight dataset

Now you'll create a synthetic medical dataset with height and weight measurements.

In [ ]:
# Set the number of samples to generate
n_samples = 1000

# Generate random heights (in cm) with realistic distribution
# Mean: 170 cm, Standard deviation: 10 cm
heights = np.random.normal(170, 10, n_samples)

# Generate random weights (in kg) with realistic distribution
# Mean: 70 kg, Standard deviation: 15 kg
weights = np.random.normal(70, 15, n_samples)

# Ensure all values are positive (remove any negative outliers)
heights = np.abs(heights)
weights = np.abs(weights)

In [ ]:
# Calculate BMI for each individual
# BMI formula: weight (kg) / (height (m))^2
# Convert height from cm to meters by dividing by 100
bmi = weights / ((heights / 100) ** 2)

In [ ]:
# Create BMI categories based on standard medical thresholds
def categorize_bmi(bmi_value):
    """
    Categorize BMI into three classes based on WHO standards
    """
    if bmi_value < 18.5:
        return "Underweight"
    elif bmi_value < 30:
        return "Normal Weight"
    else:
        return "Obese"

# Apply categorization to all BMI values
categories = [categorize_bmi(b) for b in bmi]

In [ ]:
# Create a pandas DataFrame to organize the data
df = pd.DataFrame({
    'height_cm': heights,
    'weight_kg': weights,
    'bmi': bmi,
    'category': categories
})

# Display the first few rows
print("First 5 rows of the dataset:")
print(df.head())

In [ ]:
# Display basic statistics about the dataset
print("\nDataset shape (rows, columns):", df.shape)
print("\nBasic statistics:")
print(df.describe())

In [ ]:
# Check the distribution of categories
print("\nDistribution of BMI categories:")
print(df['category'].value_counts())
print("\nPercentage distribution:")
print(df['category'].value_counts(normalize=True) * 100)

In [ ]:
# Visualize the distribution of height and weight
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot height distribution
axes[0].hist(df['height_cm'], bins=30, color='skyblue', edgecolor='black')
axes[0].set_xlabel('Height (cm)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of Height')
axes[0].grid(alpha=0.3)

# Plot weight distribution
axes[1].hist(df['weight_kg'], bins=30, color='lightcoral', edgecolor='black')
axes[1].set_xlabel('Weight (kg)')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of Weight')
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Create a scatter plot showing height vs weight colored by category
plt.figure(figsize=(10, 6))

# Plot each category with a different color
for category in df['category'].unique():
    subset = df[df['category'] == category]
    plt.scatter(subset['height_cm'], subset['weight_kg'],
                label=category, alpha=0.6, s=30)

plt.xlabel('Height (cm)', fontsize=12)
plt.ylabel('Weight (kg)', fontsize=12)
plt.title('Height vs Weight by BMI Category', fontsize=14)
plt.legend()
plt.grid(alpha=0.3)
plt.show()

## Step 5: Prepare data and train the classification model

Now you'll split the data and train a Random Forest classifier to predict BMI categories based on height and weight.

In [ ]:
# Prepare features (X) and target (y)
# Features: height and weight
X = df[['height_cm', 'weight_kg']]

# Target: BMI category
y = df['category']

print("Features shape:", X.shape)
print("Target shape:", y.shape)

In [ ]:
# Split the data into training and testing sets
# test_size=0.2 means 20% for testing, 80% for training
# random_state ensures reproducibility
# stratify ensures balanced distribution of categories in both sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Training set size:", X_train.shape[0])
print("Testing set size:", X_test.shape[0])

In [ ]:
# Initialize the Random Forest Classifier
# n_estimators: number of trees in the forest
# random_state: ensures reproducibility
model = RandomForestClassifier(
    n_estimators=100,
    random_state=42
)

print("Model initialized successfully!")

In [ ]:
# Train the model on the training data
model.fit(X_train, y_train)

print("Model training complete!")

## Step 6: Evaluate model performance

Let's assess how well the model performs on the test data.

In [ ]:
# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate accuracy score
accuracy = accuracy_score(y_test, y_pred)

print(f"Model Accuracy: {accuracy * 100:.2f}%")

In [ ]:
# Display detailed classification report
# Shows precision, recall, and F1-score for each category
print("\nDetailed Classification Report:")
print(classification_report(y_test, y_pred))

In [ ]:
# Create and visualize confusion matrix
cm = confusion_matrix(y_test, y_pred)

# Set up the plot
plt.figure(figsize=(8, 6))

# Create heatmap
sns.heatmap(
    cm,
    annot=True,  # Show numbers in cells
    fmt='d',  # Format as integers
    cmap='Blues',  # Color scheme
    xticklabels=model.classes_,  # X-axis labels
    yticklabels=model.classes_  # Y-axis labels
)

plt.title('Confusion Matrix', fontsize=14)
plt.ylabel('True Label', fontsize=12)
plt.xlabel('Predicted Label', fontsize=12)
plt.show()

In [ ]:
# Display feature importance
# Shows which features are most important for predictions
feature_importance = pd.DataFrame({
    'feature': X.columns,
    'importance': model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance)

In [ ]:
# Visualize feature importance
plt.figure(figsize=(8, 5))
plt.barh(feature_importance['feature'], feature_importance['importance'], color='teal')
plt.xlabel('Importance', fontsize=12)
plt.ylabel('Feature', fontsize=12)
plt.title('Feature Importance in BMI Classification', fontsize=14)
plt.grid(axis='x', alpha=0.3)
plt.show()

## Step 7: Save the model using joblib

Now that you have a trained model, you need to save it so you can load it later in the Gradio application.

In [ ]:
# Define the filename for the saved model
model_filename = 'bmi_classifier_model.pkl'

# Save the model to disk using joblib
joblib.dump(model, model_filename)

print(f"Model saved successfully as '{model_filename}'!")

In [ ]:
# Verify the model was saved by checking file size
import os

file_size = os.path.getsize(model_filename)
print(f"Model file size: {file_size / 1024:.2f} KB")

## Step 8: Build a Gradio app to load and use the model

This is where everything comes together! You'll create an interactive web application that loads the saved model and makes predictions.

In [ ]:
# Load the saved model from disk
loaded_model = joblib.load(model_filename)

print("Model loaded successfully!")

In [ ]:
# Define a function that takes height and weight and returns BMI category
def predict_bmi_category(height_cm, weight_kg):
    """
    Predict BMI category based on height and weight

    Args:
        height_cm: Height in centimeters
        weight_kg: Weight in kilograms

    Returns:
        String describing the predicted BMI category and calculated BMI
    """
    # Create input array in the same format as training data
    input_data = np.array([[height_cm, weight_kg]])

    # Make prediction using the loaded model
    prediction = loaded_model.predict(input_data)[0]

    # Calculate actual BMI for reference
    bmi_value = weight_kg / ((height_cm / 100) ** 2)

    # Return formatted result
    return f"Predicted Category: {prediction}\nCalculated BMI: {bmi_value:.2f}"

In [ ]:
# Test the prediction function with sample values
test_result = predict_bmi_category(170, 70)
print("Test prediction:")
print(test_result)

In [ ]:
# Create the Gradio interface for the BMI classifier
demo_app = gr.Interface(
    fn=predict_bmi_category,  # Function to call

    # Define input widgets
    inputs=[
        gr.Number(label="Height (cm)", value=170),  # Number input with default
        gr.Number(label="Weight (kg)", value=70)    # Number input with default
    ],

    # Define output widget
    outputs=gr.Textbox(lines=3, label="Prediction Result"),

    # Add title and description
    title="BMI Category Classifier",
    description="Enter height (cm) and weight (kg) to predict BMI category. This model classifies individuals as Underweight, Normal Weight, or Obese.",

    # Add example inputs for users to try
    examples=[
        [160, 45],   # Likely underweight
        [175, 70],   # Likely normal weight
        [165, 95]    # Likely obese
    ],

    # Enable interpretation to show which inputs affect the output
    allow_flagging="never"  # Disable flagging feature for simplicity
)

# Launch the application
demo_app.launch(share=False)

# Exercises

Now it's your turn to practice and extend what you've learned!

## Exercise 1: Enhance the Gradio interface with sliders

The current interface uses number input boxes. Modify the Gradio app to use **sliders** instead, which might be more intuitive for users.

**Task:** Replace the `gr.Number` inputs with `gr.Slider` widgets. Set appropriate minimum, maximum, and default values based on realistic human measurements.

In [ ]:
# your code goes here


<details>
<summary> Click here for a hint</summary>

Use `gr.Slider()` instead of `gr.Number()`.
Consider these realistic ranges:
- Height: 140-210 cm
- Weight: 30-150 kg

Use gr.Slider with parameters: minimum, maximum, value (default), label

The syntax is: `gr.Slider(minimum=min_val, maximum=max_val, value=default_val, label="Label Text")`

</details>

<details>
<summary> Click here for solution</summary>

```python
# Create enhanced Gradio interface with sliders
enhanced_demo = gr.Interface(
    fn=predict_bmi_category,
    
    # Use sliders for more intuitive input
    inputs=[
        gr.Slider(
            minimum=140,
            maximum=210,
            value=170,
            label="Height (cm)"
        ),
        gr.Slider(
            minimum=30,
            maximum=150,
            value=70,
            label="Weight (kg)"
        )
    ],
    
    outputs=gr.Textbox(label="Prediction Result"),
    
    title="BMI Category Classifier (Enhanced)",
    description="Use sliders to adjust height and weight values.",
    
    examples=[
        [160, 45],
        [175, 70],
        [165, 95]
    ],
    
    allow_flagging="never"
)

# Launch the enhanced application
enhanced_demo.launch(share=False)
```

</details>

## Exercise 2: Add prediction confidence scores

Machine learning models don't just make predictions. They also calculate confidence scores (probabilities) for each possible class. Modify the prediction function to display these probabilities.

**Task:** Update the `predict_bmi_category` function to return the prediction probabilities for all three categories.

In [ ]:
# your code goes here


<details>
<summary> Click here for a hint </summary>

Use model.predict_proba() instead of model.predict()

The `predict_proba()` method returns probabilities for each class. You can access the class names using `loaded_model.classes_`.

Loop through classes and probabilities together using `zip()` to format them nicely.

</details>

<details>
<summary> Click here for solution </summary>

```python
# Enhanced prediction function with probabilities
def predict_with_confidence(height_cm, weight_kg):
    """
    Predict BMI category and return confidence scores
    """
    # Create input array
    input_data = np.array([[height_cm, weight_kg]])
    
    # Get prediction
    prediction = loaded_model.predict(input_data)[0]
    
    # Get probabilities for all classes
    probabilities = loaded_model.predict_proba(input_data)[0]
    
    # Calculate BMI
    bmi_value = weight_kg / ((height_cm / 100) ** 2)
    
    # Format output with all probabilities
    result = f"Predicted Category: {prediction}\n"
    result += f"Calculated BMI: {bmi_value:.2f}\n\n"
    result += "Confidence Scores:\n"
    
    # Add probability for each class
    for class_name, prob in zip(loaded_model.classes_, probabilities):
        result += f"  {class_name}: {prob * 100:.2f}%\n"
    
    return result

# Create Gradio interface with the enhanced function
confidence_demo = gr.Interface(
    fn=predict_with_confidence,
    inputs=[
        gr.Slider(minimum=140, maximum=210, value=170, label="Height (cm)"),
        gr.Slider(minimum=30, maximum=150, value=70, label="Weight (kg)")
    ],
    outputs=gr.Textbox(label="Prediction with Confidence Scores"),
    title="BMI Classifier with Confidence Scores",
    description="See prediction probabilities for each category.",
    examples=[[160, 45], [175, 70], [165, 95]],
    allow_flagging="never"
)

confidence_demo.launch(share=False)
```

</details>

---

# Congratulations!

You have successfully completed this lab on building and deploying a machine learning application using Gradio! You’ve learned how to load a pre-trained model, create an interactive interface for user input, make real-time predictions, and display results clearly for end users. Along the way, you built and trained a classification model, saved it for reuse, and deployed it as a simple web app — all essential steps in bringing AI models into real-world healthcare settings. These skills form the foundation for developing practical, accessible, and trustworthy AI tools that can support clinicians and enhance patient care.

## Authors

Ramesh Sannareddy

Copyright © 2025 SkillUp. All rights reserved.
